In [1]:
import os
import numpy as np
import pandas as pd # original 1.2.3
import geopandas as gpd
from shapely.geometry import Point
import pickle
import urllib.request
from scipy import interpolate
import matplotlib.pyplot as plt

In [2]:
data_dir = '/home/simon/Documents/Bodies/data/PRIO/'

with open(f'{data_dir}prio_ucdp_full.pkl', 'rb') as file:
    df = pickle.load(file)

In [3]:
df

,gid,xcoord,ycoord,col,row,year,gwno,month,year_months_start,month_id,...,pasture_ih_observed,petroleum_y_observed,pop_gpw_sd_observed,pop_gpw_sum_observed,prec_gpcp_observed,urban_ih_observed,cmr_mean,imr_mean,mountains_mean,ttime_mean
0,239205,-97.75,76.25,165,333,1989,20.0,1,1989-01,109,...,False,False,False,False,True,False,0.0,99.0,0.000000,7253.5800
1,239205,-97.75,76.25,165,333,1989,20.0,2,1989-02,110,...,False,False,False,False,False,False,0.0,99.0,0.000000,7253.5800
2,239205,-97.75,76.25,165,333,1989,20.0,3,1989-03,111,...,False,False,False,False,False,False,0.0,99.0,0.000000,7253.5800
3,239205,-97.75,76.25,165,333,1989,20.0,4,1989-04,112,...,False,False,False,False,False,False,0.0,99.0,0.000000,7253.5800
4,239205,-97.75,76.25,165,333,1989,20.0,5,1989-05,113,...,False,False,False,False,False,False,0.0,99.0,0.000000,7253.5800
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20223211,200297,-111.75,49.25,137,279,2014,20.0,8,2014-08,416,...,False,False,False,False,False,False,0.0,44.0,0.043333,154.7922
20223212,200297,-111.75,49.25,137,279,2014,20.0,9,2014-09,417,...,False,False,False,False,False,False,0.0,44.0,0.043333,154.7922
20223213,200297,-111.75,49.25,137,279,2014,20.0,10,2014-10,418,...,False,False,False,False,False,False,0.0,44.0,0.043333,154.7922
20223214,200297,-111.75,49.25,137,279,2014,20.0,11,2014-11,419,...,False,False,False,False,False,False,0.0,44.0,0.043333,154.7922


In [2]:
data_dir = '/home/simon/Documents/Bodies/data/OD_dataframes_compiled/'

with open(f'{data_dir}df_ucdp_prio.pkl', 'rb') as file:
    prio_ucdp = pickle.load(file)

In [3]:
data_dir = '/home/simon/Documents/Bodies/data/PRIO'

with open(f'{data_dir}/full_prio_interpl.pkl', 'rb') as file:
    full_df = pickle.load(file)

In [14]:
# correction for month_id
offset = prio_ucdp['month_id'].min() -1
full_df['month_id'] = full_df['month_id'] + offset

# year cut of for ucdp and type correction
prio_ucdp = prio_ucdp[prio_ucdp['year'] <= 2014]
prio_ucdp['month_id'] = prio_ucdp['month_id'].astype(int)
prio_ucdp['month'] = prio_ucdp['month'].astype(int)

In [15]:
merge_on = list(set.intersection(set(prio_ucdp.columns),set(full_df.columns)))

#Check:
print(prio_ucdp.shape)
print(full_df.shape)

for i in merge_on:
    print(i)
    print(prio_ucdp[i].min())
    print(prio_ucdp[i].max())
    print(prio_ucdp[i].dtype)

    print(full_df[i].min())
    print(full_df[i].max())
    print(full_df[i].dtype)
    print('\n')



(20223216, 20)
(20223216, 44)
ycoord
-55.75
83.25
float64
-55.75
83.25
float64


gid
49182
249344
int64
49182
249344
int64


row
69
347
int64
69
347
int64


month_id
109
420
int64
325
636
int64


col
1
720
int64
1
720
int64


xcoord
-179.75
179.75
float64
-179.75
179.75
float64


month
1
12
int64
1
12
int64


year
1989
2014
int64
1989
2014
int64




In [16]:
prio_ucdp_full = prio_ucdp.merge(full_df, on = merge_on, how='outer')

: 

: 

In [ ]:
prio_ucdp[prio_ucdp['year'] <= 2014]['year'].shape

In [ ]:
yearly_df.columns

In [ ]:
static_df.columns

In [ ]:
full_df.columns

# OLD

In [ ]:
data_dir = '/home/simon/Documents/Bodies/data/PRIO'

with open(f'{data_dir}/static_prio_interpl.pkl', 'rb') as file:
    static_df = pickle.load(file)

In [ ]:
# old:

In [ ]:
data_dir = '/home/simon/Documents/Bodies/data/OD_dataframes_compiled/'

with open(f'{data_dir}df_ucdp_prio.pkl', 'rb') as file:
    prio_ucdp = pickle.load(file)

In [ ]:
coords_long = prio_ucdp[['gid', 'xcoord', 'ycoord']].copy() # conveniet for plotting
coords = coords_long.groupby('gid').mean()
coords.reset_index(inplace = True)

In [ ]:
# bdist3 capdist gcp_mer gcp_ppp excluded nlights_mean nlights_calib_mean 
# petroleum_y pop_gpw_sum pop_gpw_sd prec_gpcp urban_ih agri_ih pasture_ih
# barren_ih irrig_sum imr_mean cmr_mean mountain_mean  ttime_mean

location = '/home/simon/Documents/Bodies/data/PRIO'
path_yearly = location + '/PRIO-GRID Yearly Variables for 1989-2014 - 2022-09-08.csv' #https://grid.prio.org/#/download 
path_static = location + '/PRIO-GRID Static Variables - 2022-09-08.csv' #https://grid.prio.org/#/download 

#yearly_df = pd.read_csv(path_yearly)
#yearly_df = yearly_df.merge(coords, on='gid', how='left') # for plotting. faster to scatter then geopandas plot

static_df = pd.read_csv(path_static) # already have coords. You could get them from here..
#static_df = static_df.merge(coords, on='gid', how='left') # for plotting. faster to scatter then geopandas plot



In [ ]:
data_dir = '/home/simon/Documents/Bodies/data/PRIO'

with open(f'{data_dir}/yearly_prio_interpl.pkl', 'rb') as file:
    yearly_df = pickle.load(file)

In [ ]:
def explode_static(static_df, yearly_df):
    
    n_months = 12
    years = sorted(yearly_df['year'].unique())
    n_gid = yearly_df['gid'].unique().shape[0] # number of groups - could be from either static or yearly. Same
    n_years = yearly_df['year'].unique().shape[0] # number of years

    df = static_df.copy()

    df['year'] = [years] * df.shape[0]
    df['month'] = [list(np.arange(1, n_months+1))] * df.shape[0]

    df = df.explode('year')
    df = df.explode('month')

    df.reset_index(inplace=True, drop= True)
    
    df['month_id'] = list(np.arange(1,n_years * n_months+1)) * n_gid 

    return(df)

In [ ]:
static_df = explode_static(static_df, yearly_df)
merge_on = list(set.intersection(set(yearly_df.columns), set(static_df.columns)))
print(merge_on)

In [ ]:
yearly_df = yearly_df.merge(static_df, on=merge_on, how='outer', copy = False)

In [ ]:
full_df

In [ ]:
data_dir = '/home/simon/Documents/Bodies/data/PRIO'
full_df.to_pickle(f'{data_dir}/full_prio_interpl.pkl')
static_df.to_pickle(f'{data_dir}/static_prio_interpl.pkl')

In [ ]:
yearly_df

In [ ]:
yearly_df.merge(df, on=['gid', 'year', 'month_id'])

In [ ]:
yearly_df.columns

In [ ]:
df.columns

In [ ]:
set.intersection(set(yearly_df.columns), set(df.columns))

In [ ]:
df = yearly_df.copy()

In [ ]:
df.sort_values(['gid', 'year'], inplace=True)

In [ ]:
def expl_interpl_extrapl(data, feature_list):
    
    n_months = 12
    n_gid = data['gid'].unique().shape[0] # number of groups
    n_years = data['year'].unique().shape[0] # number of years

    # list of lists with temporal sub unites. eg. months
    months = [list(np.arange(1, n_months+1))] * data.shape[0] # list of lists with temporal sub unites. eg. months

    # append the colum
    data['month'] = months 

    # explode the df in reference to the new col and drop the ond index
    data = data.explode('month').reset_index(drop=True) 

    # we'll only keep the original values in the first tempoeal sub unit. Interpolate rest.
    data.loc[data['month'] != 1, feature_list] = np.nan 


    # number of year x number of months. And then number of groups. +1 becaus Views month_id starts at 1
    data['month_id'] = list(np.arange(1,n_years * n_months+1)) * n_gid 

    # to be used below
    obs_features_list = []

    for f in feature_list:
        observed_col_name = f'{f}_observed'

        # an identifier for the non-interpolated values
        data[observed_col_name] = data.loc[:,f].notna() 
        obs_features_list.append(observed_col_name)

    # interpolate and extrapolate
    data = data.groupby('gid').apply(lambda x: x.interpolate(method='linear', axis = 0, limit_direction = 'both', fill_value='extrapolate'))


    # clip at group observed min and max for each feature.
    for f, o in list(zip(feature_list, obs_features_list)):
        data[f] = data.groupby('gid').apply(lambda x: np.clip(x[f], x[x[o] == True][f].min(), x[x[o] == True][f].max())).reset_index(drop=True)

    return(data)

In [ ]:
feature_list = list(df.columns[2:-2])
new_df = expl_interpl_extrapl(df, feature_list)

In [ ]:
new_df.to_pickle('test_interpolate.pkl')

In [ ]:
df_no_nan = new_df.copy()
df_no_nan.fillna(0, inplace=True)

In [ ]:
df_no_nan.to_pickle('/home/simon/Documents/Bodies/data/PRIO/yearly_prio_interpl.pkl')

In [ ]:
df_no_nan['month_id'].unique() 

In [ ]:
def plot_maps(df):

    map_dir = '/home/simon/Documents/Bodies/figures/maps/PRIO_timelaps/'

    for feature in df.columns[2:3]:#[2:-20]:

        feature_dir = os.path.join(map_dir, feature)
        os.makedirs(feature_dir, exist_ok = True)

        for month_id in df['month_id'].unique():

            x = df.loc[new_df['month_id'] == month_id, 'xcoord'] 
            y = df.loc[new_df['month_id'] == month_id, 'ycoord'] 
            c = df.loc[new_df['month_id'] == month_id, feature] 

            plt.figure(figsize=[28,12])
            plt.scatter(x, y, c = c, cmap = 'rainbow', marker = 's', s = 2, vmin= c.min(), vmax=c.max())
        
            fig_title = f'{feature}_{str(month_id).zfill(3)}'

            plt.title(fig_title)
            plt.savefig(feature_dir + '/' + fig_title + '.JPG', bbox_inches="tight")
            plt.cla() # idk if this is also needed..
            plt.close('all') # so they do not display or take up mem


In [ ]:
plot_maps(df_no_nan)

In [ ]:
'1'.zfill(3)

In [ ]:
feature = 'sks'
month_id = 2
print(f'{feature}_{str(month_id).zfill(3)}')

In [ ]:
list(np.arange(1,10 * 12+1))

# OLD

In [ ]:
# SEEMS TO WORK!!!
df_interpolate = df.groupby('gid').apply(lambda x: x.interpolate(method='linear', axis = 0, limit_direction = 'both'))

In [ ]:
df_interpolate

- have a dumme of original values
- plot samples to see.
- se om du kan gører det til mdr. pandas explode...

In [ ]:
test_gid = 180640
df_interpolate[df_interpolate['gid'] == test_gid]['agri_ih']

In [ ]:
test_gid = 180640
df[df['gid'] == test_gid]['agri_ih']

In [ ]:
# prof of concept

In [ ]:
ex_test = pd.DataFrame({'value': [60,70,90,140,300,120], 'year' : [1990, 1991, 1992,1990, 1991, 1992], 'group' : [1,1,1,2,2,2]}) # some df ith vlaues, temporal unites and groups
ex_test

In [ ]:
q = [[1,2]] * ex_test.shape[0] # list of lists with temporal sub unites. eg. months
ex_test['quarter'] = q # append the colum
ex_test = ex_test.explode('quarter').reset_index(drop=True) # explode the df in reference to the new col and drop the ond index
ex_test.loc[ex_test['quarter'] != 1, 'value'] = np.nan # we'll only keep the original values in the first tempoeal sub unit. Interpolate rest.
ex_test


In [ ]:
nt = ex_test['year'].unique().shape[0] * ex_test['quarter'].unique().shape[0] # number of time unites
gn = ex_test['group'].unique().shape[0] # number of groups

ex_test['t_id'] = list(range(nt)) * gn # uniq temporal idenitfier

In [ ]:
ex_test

In [ ]:
#ex_test = ex_test.groupby('group').apply(lambda x: x.interpolate(method='polynomial', order = 2, axis = 0, limit_direction = 'both'))

#ex_test = ex_test.groupby('group').apply(lambda x: interpolate.interp1d(x, ex_test['t_id'].unique(), fill_value='extrapolate'))
#f = interpolate.interp1d(x, ex_test['t_id'].unique(), fill_value='extrapolate')

ex_test = ex_test.groupby('group').apply(lambda x: x.interpolate(method='linear', axis = 0, limit_direction = 'both', fill_value='extrapolate'))


ex_test

In [ ]:
for i in range(3):
    plt.plot(ex_test.loc[ex_test['group'] == i, 't_id'], ex_test.loc[ex_test['group'] == i, 'value'])
plt.show()

In [ ]:
df.groupby('gid').apply(lambda x: x.interpolate(method='linear', axis = 0, limit_direction = 'both'))

In [ ]:
df = yearly_df.copy()
df['year'] = pd.to_datetime(df['year'], format='%Y')
df.index = df['year']
df.sort_index(inplace= True)
del df['year']

In [ ]:
df

In [ ]:
# df_interpol = df.groupby('gid').resample('Y').mean()
#df_interpol = df.groupby('gid').apply(lambda x: x.interpolate(method='linear'))


In [ ]:
df_interpol = df.groupby('gid').apply(lambda x: x.interpolate(method='linear', axis = 0))
# df_interpol = df.groupby('gid').apply(lambda x: x.interpolate(method='linear', axis = 0)).bfill().ffill()


In [ ]:
df_interpol.shape == df.shape

In [ ]:
dum = pd.DataFrame({'time': [1,2,3,4,5,1,2,3,4,5],'group': [1,1,1,1,1,2,2,2,2,2], 'value1': [np.nan,10,np.nan,20,np.nan,1,np.nan,np.nan,4,np.nan],'value2': [np.nan,100,np.nan,200,np.nan,10,np.nan,np.nan,40,np.nan]})
dum

In [ ]:
dum.groupby('group').apply(lambda x: x.interpolate(method='linear', axis = 0, limit_direction = 'both')) 

In [ ]:
agri_ih = df.groupby('gid')['agri_ih'].apply(lambda x: x.interpolate(method='linear')) #.bfill()

In [ ]:
test_gid = 180640
df_interpol[df_interpol['gid'] == test_gid]['agri_ih'].sort_index()

In [ ]:
test_gid = 180640
df[df['gid'] == test_gid]['agri_ih'].sort_index()

In [ ]:
#df_interpol_done = df_interpol.interpolate(method='linear', axis=0).copy()
#df_interpol_done = df.interpolate(method='linear', axis=0).copy()
df_interpol_done = df_interpol.ffill().bfill().copy()


In [ ]:
test_gid = 180640
df_interpol_done[df_interpol_done['gid'] == test_gid]['agri_ih'].sort_index()

In [ ]:
test_gid = 180640
df_interpol_done[df_interpol_done['gid'] == test_gid]['agri_ih'].sort_index()

In [ ]:
test_gid = 180640
df_interpolate[df_interpolate['gid'] == test_gid]['agri_ih']


In [ ]:
df_interpol = df.groupby('gid').bfill()
df_interpol['gid'] = df['gid']

In [ ]:
df_interpol_done = df_interpol.interpolate(method='linear', axis=0, limit_area = 'outside').copy()
df_interpol_done.head(4)

In [ ]:
test_gid = 180640
df_interpol[df_interpol['gid'] == test_gid]['agri_ih'].sort_index()

In [ ]:
test_gid = 180640
df_interpol_done[df_interpol_done['gid'] == test_gid]['agri_ih']

In [ ]:
test_gid = 180640
df_interpol_done[df_interpol_done['gid'] == test_gid]['agri_ih']

In [ ]:
test_gid = 180640
test_df = yearly_df.copy()
test_df = test_df[test_df['gid'] == test_gid][['agri_ih', 'year']].sort_values('year').set_index('year')
#test_df.interpolate(method='linear', axis=0, inplace=True)
test_df

In [ ]:
test_gid = 180640
test_df = yearly_df.copy()
test_df = test_df[test_df['gid'] == test_gid][['agri_ih', 'year']].sort_values('year').set_index('year')
test_df.interpolate(method='linear', axis=0, inplace=True)
test_df

In [ ]:
yearly_df.head()

In [ ]:
test_gid = 180640
test_df = yearly_df[yearly_df['gid'] == test_gid][['agri_ih', 'year']].sort_values('year').set_index('year')
test_df.interpolate(method='linear', axis=0, inplace=True)
test_df

In [ ]:
sub_yearly_df = yearly_df[yearly_df['year'] == 2000]

for col in sub_yearly_df.columns:
    print(col)
    plt.figure(figsize=[28,12])
    plt.scatter(sub_yearly_df['xcoord'], sub_yearly_df['ycoord'], c = sub_yearly_df[col], cmap = 'rainbow', marker = 's', s = 2)
    plt.show()

In [ ]:
#plt.figure(figsize = [14,8])
fig, ax = plt.subplots(figsize = (20,12))
ax.set_aspect('equal')

plt.scatter(sub_prio_ucdp['xcoord'], sub_prio_ucdp['ycoord'], c = np.log(sub_prio_ucdp['best']+1), cmap = 'rainbow', marker = 's', s = 560)
plt.scatter(bodies['longitude_full'].unique()[:-1], bodies['latitude_full'].unique()[:-1], color = 'black', marker = 'x', s = 235)
sub_world.plot(facecolor='none', edgecolor='black', ax=ax)


plt.show()

In [ ]:
yearly_df[yearly_df['year'] == 2000].info()

In [ ]:
static_df.info()

In [ ]:
def get_gwno():

    location = '/home/simon/Documents/Bodies/data/PRIO'
    path_gwno = location + '/PRIO-GRID Yearly Variables for 1989-2014 - 2022-06-16.csv' #https://grid.prio.org/#/download Simply contains the GW contry code and the meta.

    # why not just go 1989 - 2019 like ucdp...

    gwno = pd.read_csv(path_gwno)

    return gwno




 # Why some features that might be interesting are not here:

Fra Uppsala:

- event_clarity 
- date_prec 
- type_of_violence

Fra Prio


In [ ]:
data_dir = '/home/simon/Documents/Bodies/data/OD_dataframes_compiled/'

with open(f'{data_dir}bodies_df_exposure.pkl', 'rb') as file:
    df = pickle.load(file)

In [ ]:
df